<a href="https://colab.research.google.com/github/bhanup6663/COMP691_DL/blob/Niloofar/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#Cell1
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import random
from sklearn.model_selection import train_test_split


In [34]:
#Cell 2
random.seed(42)
np.random.seed(42)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [35]:
#Cell 3

def filter_and_sample_by_class(dataset, classes, samples_per_class):
    indices = []
    for cls in classes:
        cls_indices = [i for i, (_, label) in enumerate(dataset) if label == cls]
        sampled_indices = random.sample(cls_indices, samples_per_class)
        indices.extend(sampled_indices)
    return Subset(dataset, indices)

classes = random.sample(range(10), 2)
filtered_train_dataset = filter_and_sample_by_class(train_dataset, classes, 25)
filtered_test_dataset = filter_and_sample_by_class(test_dataset, classes, 1000)

train_loader = DataLoader(filtered_train_dataset, batch_size=len(filtered_train_dataset), shuffle=True)
test_loader = DataLoader(filtered_test_dataset, batch_size=len(filtered_test_dataset), shuffle=False)


In [36]:
#Cell 4


def extract_features_and_labels(loader):
    for images, labels in loader:
        features = images.view(images.size(0), -1).numpy()
        labels = labels.numpy()
        return features, labels

x_train, y_train = extract_features_and_labels(train_loader)
x_test, y_test = extract_features_and_labels(test_loader)

pipeline = Pipeline([
    ('lda', LDA(n_components=1)),
    ('svc', SVC(random_state=42))
])

param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['linear']
}

grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, verbose=3)

grid_search.fit(x_train, y_train)

print(f"Best Parameters: {grid_search.best_params_}")
best_model = grid_search.best_estimator_
predictions = best_model.predict(x_test)
validation_accuracy = accuracy_score(y_test, predictions) * 100


Fitting 3 folds for each of 3 candidates, totalling 9 fits
Best Parameters: {'svc__C': 1, 'svc__kernel': 'linear'}


In [37]:
# #Cell 5
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

best_accuracy = 0
best_epoch = -1
best_model = None

for epoch in range(10):  # Simulating epochs
    print(f"Epoch {epoch + 1}/10")

    grid_search.fit(X_train, y_train)


    current_model = grid_search.best_estimator_
    predictions = current_model.predict(X_val)
    current_accuracy = accuracy_score(y_val, predictions)
    print(f"Validation Accuracy: {current_accuracy:.2f}%")

    if current_accuracy > best_accuracy:
        best_accuracy = current_accuracy
        best_model = current_model
        best_epoch = epoch

print(f"Best Epoch: {best_epoch + 1}")
print(f"Best Validation Accuracy: {best_accuracy:.2f}%")

predictions = best_model.predict(x_test)
test_accuracy = accuracy_score(y_test, predictions) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(classification_report(y_test, predictions))

Epoch 1/10
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Validation Accuracy: 0.90%
Epoch 2/10
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Validation Accuracy: 0.90%
Epoch 3/10
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Validation Accuracy: 0.90%
Epoch 4/10
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Validation Accuracy: 0.90%
Epoch 5/10
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Validation Accuracy: 0.90%
Epoch 6/10
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Validation Accuracy: 0.90%
Epoch 7/10
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Validation Accuracy: 0.90%
Epoch 8/10
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Validation Accuracy: 0.90%
Epoch 9/10
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Validation Accuracy: 0.90%
Epoch 10/10
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Validation Accuracy: 0.90%
Best Epoch: 1
Best Validation